# Company Brochure

Uses a given url to gather information about a company and creates a brochure with it.
- Scrape the url and for contents and useful links
- Ask AI Agent to trim the links and return only useful links
- Scrape the content of the useful links
- Concatenate all the contents together
- Ask AI Agent to create a brochure with the contents
- Display the brochure

In [ ]:
import os
os.chdir('..')

In [ ]:
from IPython.display import Markdown, display
from scraper.scraper import SoupScraper, Website
from utils.utils import load_env_variables
from week_one.company_brochure import MessageCreator
from agents.agents import Agent, OpenAIAgent

Get details required to create the brochure from the website url

In [ ]:
def get_useful_links(website: Website, agent: Agent, model: str):
    msgCreator = MessageCreator()
    messages = msgCreator.create_links_message(website)
    response = agent.respond_with_json(model, messages)
    return response

In [ ]:
def scrape_url(url: str) -> Website:
    scraper = SoupScraper(url)
    return scraper.get_website_content()

In [ ]:
def get_website_content(website: Website) -> str:
    return f"Webpage Title:\n{website.title}\nWebpage Contents:\n{website.body}\n\n"

In [ ]:
def get_details(url: str, agent: Agent, model: str) -> str:
    result = "Landing page:\n"
    website = scrape_url(url)
    result += get_website_content(website)
    
    links = get_useful_links(website, agent, model)
    for link in links["links"]:
        if link:
            result += f"\n\n{link['type']}\n"
            website = scrape_url(link['url'])
            result += get_website_content(website)
        
    return result

In [ ]:
def create_brochure(company_name: str, url: str, agent: Agent, model: str):
    msgCreator = MessageCreator()
    messages = msgCreator.create_brochure_message(
        company_name, get_details(url, agent, model)
    )
    response = agent.respond_with(model, messages)
    display(Markdown(response))

In [ ]:
company_name = "HuggingFace"
company_url = "https://huggingface.co"

In [ ]:
load_env_variables()  # loads open ai key

In [ ]:
openai_agent = OpenAIAgent()
model = "gpt-4o-mini"
create_brochure(company_name, company_url, openai_agent, model)